### 분석 타겟 컬럼

- Life_expectancy: 기대수명

### 1번 (유아, 청소년 관련 변수)

- Infant_deaths: 영아 사망 수
- Under_five_deaths: 5세 이하 사망 수
- Polio: 소아마비 예방접종률
- Thinness_five_nine_years: 5-9세의 저체중 비율
- Thinness_ten_nineteen_years: 10-19세의 저체중 비율

---
colorcode :'viridis'

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('./data/Life-Expectancy-Data-Averaged.csv')
df

- list01 : 분석 타겟을 포함한 1번 카테고리의 컬럼 리스트
- df_1st : 새로 분류한 데이터
- val_1st : 분석 타겟을 제외한 컬럼 리스트

In [14]:
# 1번 카테고리 따로 분류

list01 = ['Life_expectancy', 'Infant_deaths', 'Under_five_deaths', 'Polio', 'Thinness_five_nine_years', 'Thinness_ten_nineteen_years']
val_1st = ['Infant_deaths', 'Under_five_deaths', 'Polio', 'Thinness_five_nine_years', 'Thinness_ten_nineteen_years']

df_1st = df[list01]

In [ ]:
df_1st.info()

#### 각 요소간의 상관관계 평가

In [ ]:
corr_matrix = df_1st.corr(numeric_only=True)
corr_matrix

sns.heatmap(corr_matrix, annot=True, cmap='viridis')
plt.show()

- 0에 가까울수록 상관관계가 없고, 1은 양의 상관관계, -1은 음의 상관관계를 보여줌.
#### Life_expectancy(기대수명)과 다른 요소들의 관계:
- Infant_deaths와 Under_five_deaths: 상관계수 -0.93
    - 매우 강한 음의 상관관계.
    - 유아 사망률이나 5세 미만 사망률이 높을수록 기대수명이 낮아진다는 것을 보여줌.
- Polio: 상관계수 0.68
    - 중간 정도의 양의 상관관계.
    - 소아마비 백신 접종률이 높을수록 기대수명이 증가하는 경향.
- Thinness_five_nine_years 및 Thinness_ten_nineteen_years: 상관계수 각각 -0.49, -0.51
    - 약한 음의 상관관계.
    - 저체중율이 높아질수록 기대수명이 감소하는 경향이 있지만, 앞의 변수들에 비해 영향력이 적음.
#### Infant_deaths와 Under_five_deaths:
- 상관계수 0.99
- 거의 완벽한 양의 상관관계.
- 이는 유아 사망률과 5세 미만 사망률이 서로 밀접하게 연결되어 있다는 것을 의미함.
- 실제로 유아 사망은 5세 미만 사망의 주요 구성 요소일 가능성이 큼.
#### Thinness_five_nine_years와 Thinness_ten_nineteen_years:
- 상관계수 0.99
- 거의 완벽한 양의 상관관계.
- 5~9세의 저체중은 10~19세의 저체중까지 밀접한 관계가 있음을 보여줌
#### Polio와 다른 변수들:
- Infant_deaths 및 Under_five_deaths와는 각각 -0.79로 음의 상관관계.
- 소아마비 백신 접종률이 낮은 지역에서 유아 및 5세 미만 사망률이 높다는 것을 시사함.
- Thinness 변수들과는 약한 음의 상관관계(-0.37).
- 백신 접종률이 저체중율과도 연관이 있을 수 있지만, 관계는 약함.

####

In [ ]:
fig, ax = plt.subplots(figsize=(15,10), ncols=3, nrows=2)

columns = val_1st


for i, col_name in enumerate(columns):
    row = i // 3
    col = i % 3
    ax_temp = ax[row][col]

    sns.regplot(data=df, x = col_name, y = 'Life_expectancy', ax = ax_temp)

- 실제로 산점도를 그려보았을때 영유아기 사망률과 소아마비 백신 접종률은 기대수명과 밀접한 연관이 있지만 저체중 요소는 큰 연관성을 보여주지 못함

In [ ]:
# 비슷한 변수 합치기
df['Child_deaths'] = df['Infant_deaths'] + df['Under_five_deaths']
df = df.drop(['Infant_deaths', 'Under_five_deaths'], axis=1)
df['Thinness_youth'] = df['Thinness_five_nine_years'] + df['Thinness_ten_nineteen_years']
df = df.drop(['Thinness_five_nine_years', 'Thinness_ten_nineteen_years'], axis=1)
df

In [ ]:
# 박스플롯 그리기
columns = ['Life_expectancy','Child_deaths', 'Polio', 'Thinness_youth']

plt.figure(figsize=(15, 15))
for i, column in enumerate(columns, 1):
    plt.subplot(2, 2, i)  
    sns.boxplot(data=df, y=column, color="lightblue")
    plt.title(column)
    plt.tight_layout()

plt.show()

In [ ]:
sns.barplot(data=df, y='Polio', hue='Region')
plt.show()

In [ ]:
sns.barplot(data=df, y='Life_expectancy', hue='Region')
plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 1. 데이터 준비
# 예: df는 데이터프레임, y는 종속 변수, X는 독립 변수들
X = df[['Child_deaths', 'Polio', 'Thinness_youth']]
y = df['Life_expectancy']

# 2. 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 3. 회귀 모델 생성 및 학습
lr = LinearRegression()  # 선형 회귀 모델 생성
lr.fit(X_train, y_train)  # 학습 진행

# 4. 모델 평가
# 테스트 데이터에 대한 예측
y_pred = lr.predict(X_test)

# 성능 지표 계산
mse = mean_squared_error(y_test, y_pred)  # 평균 제곱 오차
r2 = r2_score(y_test, y_pred)  # R-squared (설명력)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

# 5. 회귀 계수 출력
print("회귀 계수 (Coefficients):", lr.coef_)
print("절편 (Intercept):", lr.intercept_)

